In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import baseGrafo
import random
import math
import os
import random
import statistics
from utils import ler_grafo_dimacs, ler_solucoes_otimas, salvar_resultados

In [ ]:
baseGrafo.grafoImport(30)

In [ ]:
pasta_grafos = "grafos"
resultados_dir = "resultados"
os.makedirs(resultados_dir, exist_ok=True)

solucoes_otimas = ler_solucoes_otimas("solucoes_otimas.csv")

G = ler_grafo_dimacs(os.path.join("grafos", "r250.5.col"))
print(G.number_of_edges())
nx.draw(G, with_labels=True)      

In [ ]:
G = nx.read_edgelist("grafoBase.csv", delimiter=",")
G = nx.relabel_nodes(G, lambda x: int(x))
print(list(G.neighbors(2)))

nx.draw_circular(G, with_labels=True)

In [ ]:
def custo(coloring):
        # Conta o número de conflitos (nós adjacentes com mesma cor)
        conflitos = sum(1 for u, v in G.edges() if coloring[u] == coloring[v])
        return conflitos

def simulated_annealing_coloring(G, max_iter=500000, i_temp=12400, cooling_rate=0.995):
   # Inicializa coloração aleatória
    coloring, n_colors = baseGrafo.heuristicaGulosa(G)
    
    current_custo = custo(coloring)
    best_coloring = coloring.copy()
    best_custo_i = current_custo # Pego dados da solução inicial feita pela heuristica

    for node in coloring:
        if coloring[node] >= n_colors:
            coloring[node] = random.randint(0, n_colors - 1)
    
    current_custo = custo(coloring) 
    best_custo = current_custo
    print("solução inicial tem", n_colors+1, "cores")
    print("best custo", best_custo)
    temp = i_temp
    f_colors = n_colors
    flag = False
    checkpoint = 0
    for i in range(max_iter):
        percentual = 0.05 # até 10% dos nós
        k = max(1, int(G.number_of_nodes() * percentual * (temp / i_temp)))

        nodes = random.sample(list(G.nodes()), k)

        # Armazenar cores antigas para possível rollback
        old_colors = {node: coloring[node] for node in nodes}

        # Aplica novas cores aleatórias
        for node in nodes:
            nova_cor = random.randint(0, n_colors - 1)
            coloring[node] = nova_cor
        
        new_custo = custo(coloring)

        dif = new_custo - current_custo
        
        if dif < 0 or random.random() < math.exp(-dif / temp):
            # Aceita nova solução
            current_custo = new_custo
            if new_custo < best_custo:
                best_coloring = coloring.copy()
                best_custo = new_custo
                print("achou melhor custo", best_custo, " temperatura", temp)
                checkpoint = i
        else:
            # Reverte mudança
            for node in nodes:
                coloring[node] = old_colors[node]

        # Opcional: parar se custo for 0 e tentar com menos cores(sem conflitos)
        if best_custo == 0:
            best_coloring_i = coloring.copy()
            print("convergiu em 0, com", n_colors,"cores", "na iteracao", i)
            f_colors =  n_colors # qtd de cores de quando houve o encontro da resposta viavel
            n_colors -= 1
            best_custo_i = best_custo # numero de convergencia
            for node in coloring:
                if coloring[node] >= n_colors:
                    coloring[node] = random.randint(0, n_colors - 1)
            
            print("temperatura:", temp)
            best_custo = custo(coloring)
            temp = i_temp
            current_custo = best_custo
            
            flag = True

        if i == max_iter-1 and flag == False:
            best_coloring_i = best_coloring
            best_custo_i = best_custo
            print("bateu o ponto")
            print("temperatura:", temp)
        
        # Atualiza temperatura
        if(i%100000 == 0):
            print("temperatura atual:", temp)
            #if(checkpoint - i > 50000):
               # temp = i_temp/4
        temp *= cooling_rate

    return best_coloring_i, best_custo_i, f_colors

# Exemplo de uso
coloring, custo_final, cores= simulated_annealing_coloring(G)

print('Coloração final:', coloring)
print(cores, 'cores')
print('Conflitos restantes:', custo_final)

In [ ]:
def desenhar_grafo_colorido(G, coloring):
    # Cria uma lista de cores para os nós com base na coloração fornecida
    node_colors = [coloring[node] for node in G.nodes()]
    for u, v in G.edges():
        if node_colors[u] == node_colors[v]:
            print("bateu", u ,"e", v)

    # Desenhar o grafo
    pos = nx.spring_layout(G, seed=42)  # layout bonito e fixo
    nx.draw(
        G,
        pos,
        with_labels=True,
        node_color=node_colors,
        cmap=plt.cm.tab10,  # até 10 cores distintas
        node_size=500,
        edge_color='gray',
        font_color='white'
    )
    plt.title("Grafo com Coloração")
    plt.show()

desenhar_grafo_colorido(G, coloring)



In [ ]:
print(list(G.neighbors(5)))
print(coloring[5])
for node in G.nodes:
    if coloring[node] == 1:
        print(node,coloring[node])